In [1]:
import sys
sys.path.append("/home/tagore/repos/ai/scripts")
import os
import time
import ollama
import chromadb
import torch
import h5py
from torch  import cuda
from transformers import AutoTokenizer, AutoModel
import generate_embeddings as ge

In [2]:
documents_directory = '/home/tagore/repos/ai/data/processed/texts/Bookshelf_NBK279690.pdf'
collection_name = "blast_db_unprocessed"
database_file = '/home/tagore/repos/ai/data/processed/embeddings/chrome_db_unprocessed'

In [3]:
client = chromadb.PersistentClient(path=database_file)
collection = client.get_collection(name=collection_name)

In [6]:
blast_dna_questions = ["What is word size parameter in BLAST?",
"How to get the results of BLASTP in XML format?",
"How to perform a BLAST on a specific taxonomic group?",
"What parameters do I use to perform BLAST with epitopes smaller than 10 amino acids?",
"Which kind of databases can be searched with BLASTX?"]

In [8]:
# an example prompt
prompt = blast_dna_questions[0]

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=3
)
data = results['documents'][0]

['Table continued from previous page.\nBLAST_USAGE_REPORT Specifies  whether or not usage information should be returned to the NCBI. Set this variable \nto false to disable this feature.\nControlling concatenation of queries\nAs described above, BLAST+ works more efficiently  if it scans the database once for multiple queries. This \nfeature is knows as concatenation. Unfortunately, for some searches the concatenation values are not optimal, \ntoo many queries are searched at once, and the process can consume too much memory. For applications besides \nBLASTN (which uses an adaptive approach), it is possible to control these values by setting the BATCH_SIZE \nenvironment variable. Setting the value too low will degrade performance dramatically, so this environment \nvariable should be used with caution.\nMemory usage\nThe BLAST search programs can exhaust all memory on a machine if the input is too large or if there are too \nmany hits to the BLAST database. If this is the case, pleas

In [9]:
def generate_prompt(question, context):
    template = """You need to give instructions on how we can use specific software.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    keep the answer concise.
    user
    Question: {question} 
    Context: {context} 
    Do not say according to the text. just give the answer, no comment."""
    return template.format(question=question, context=context)

In [10]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3",
  prompt=generate_prompt(prompt, data)
)

print(output['response'])

The word size parameter in BLAST is an integer that represents the length of initial exact match. Its default value varies depending on the task:

* megablast: 28
* dc-megablast: 11
* blastn: 11
* blastn-short: 7


In [26]:
model_name="dmis-lab/biobert-base-cased-v1.2"
# Initialize BioBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
em = ge.get_embeddings(prompt[4], model, tokenizer)
results = collection3.query(
  query_embeddings=em.tolist(),
  n_results=3
)

NameError: name 'collection3' is not defined